# Project Prototype: Agentic RAG System with Nursing Handbooks and Transes as Knowledge Base

The goal of this project is to create an **Agentic RAG-based system** that helps nursing students retrieve relevant information from nursing handbooks and their personal study notes. This system will augment the responses with context from the personal notes, making it more personalized and adaptive to the user's learning.

## Flow Overview

The project involves two primary stages:
1. **Prepopulating the Vector DB** (embedding the nursing handbooks and personal notes into a database)
2. **RAG Modeling** (retrieving relevant information and augmenting responses using both the handbooks and personal notes)

Additionally, there will be an **Agentic Layer** that intelligently routes queries to the appropriate source (nursing handbooks or personal study notes).

### Technologies to use

- **Docling**: For parsing the nursing handbooks and personal notes
- **OpenAI Embedding Model (large)**: For embedding both nursing handbooks and personal study notes.
- **ChromaDB**: For storing and querying the embeddings.
- **Deepseek LLM**: For augmenting responses based on the retrieved content.
- **Pydantic AI**: For AI agent intelligently routing queries between the nursing handbooks and personal notes. 
- **Pydantic**: For type safety and data validation.
- **FastAPI**: For building the API.
- **Docker**: For containerization and deployment.
- **Pydantic Graphs**: For workflow pipelines

## Initialization

In [ ]:
import os
import joblib
import threading
import asyncio
import chromadb
import nanoid
from pathlib import Path
import concurrent.futures
from rich import print
from docling.document_converter import DocumentConverter
from abc import ABC, abstractmethod
import chromadb.utils.embedding_functions as embedding_functions
from docling_core.transforms.chunker.hierarchical_chunker import DocChunk
from pydantic import BaseModel
from docling.chunking import HybridChunker

from dotenv import load_dotenv

load_dotenv()
knowledge_base_raw_path = './data/knowledge-base/raw/'
knowledge_base_pickled_path = './data/knowledge-base/pickled/'

### Stage 1: Prepopulate the Vector DB

In this stage, we process nursing handbooks and personal study notes, embedding them into a vector database for efficient retrieval during RAG modeling.

#### Steps:

1. **Upload Nursing Handbooks**:
   - Upload nursing handbooks in popular formats like PDF or DOCX.
   - These documents may include textbooks on topics like **medical-surgical nursing, pharmacology, pediatric nursing**, and other specialized nursing areas.

2. **Parsing the documents and retaining the page metadata**:
   - Use **Docling** to convert nursing handbooks and personal notes docling document.
   - The conversion ensures that the pdf metadata are preserved, making it easier for the AI to process.

3. **Generate Embeddings**:
   - Use **OpenAI's large embedding model** to convert the nursing handbooks and personal notes into embeddings. These embeddings will capture the semantic meaning of each section, allowing for efficient similarity-based searches.
   - Both the nursing handbooks and the personal study notes will be embedded into the vector database.

4. **Save to Vector DB (ChromaDB)**:
   - Store the generated embeddings in **ChromaDB** for fast retrieval during RAG modeling.
   - The vector database will allow the system to quickly access the most relevant information when a query is made.

**Note**: Converting and embedding long documents like nursing handbooks may take some time (e.g., **15-30 minutes** per document).



In [154]:
# List the raw files to convert
os.listdir(knowledge_base_raw_path)

['Handbook of Clinical Nursing_ Medical-Surgical Nursing -- Joyce Fitzpatrick -- 2018 -- Springer Publishing Company -- 9780826130785 -- 26f2533f396508e653d45e0e76aadc53 -- Anna’s Archive.pdf',
 'NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL FUNCTION.pdf']

In [155]:
document_converter = DocumentConverter()

knowledge_base_raw_files = os.listdir(knowledge_base_raw_path)
knowledge_base_pickled_files = os.listdir(knowledge_base_pickled_path)

for file in knowledge_base_raw_files:
	file_path = os.path.join(knowledge_base_raw_path, file)
	file_name = os.path.splitext(file)[0]

	# Check if not file 
	if not os.path.isfile(file_path):
		continue

	# Check if file is already pickled
	if f"{file_name}.docling" in knowledge_base_pickled_files:
		print(f"File `{file[:97]}`... is already pickled.")
		continue

	# Pickling the document to retain the metadata (there is currently no way to get the paging metadata from exported md)
	print(f"Pickling file: `{file[:97]}`...")
	conversion_result = document_converter.convert(file_path)
	
	joblib.dump(
		conversion_result.document, 
		os.path.join(knowledge_base_pickled_path, f"{file_name}.docling"),
		compress=3
	)

File `Handbook of Clinical Nursing_ Medical-Surgical Nursing -- Joyce Fitzpatrick -- 2018 -- Springer P`... is 
already pickled.

File `NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL FUNCTION.pdf`... is already 
pickled.

#### Create the collections

In [157]:
chroma_client = await chromadb.AsyncHttpClient(host="localhost", port="8001")

Using the OpenAI text-embedding-3-small model

In [159]:
embed_fn_openai = embedding_functions.OpenAIEmbeddingFunction(
	api_key=os.getenv("OPENAI_API_KEY"),
	model_name=os.getenv("OPENAI_EMBEDDING_MODEL"),
)

Creating the collections

In [160]:
collection_base_documents = await chroma_client.get_or_create_collection(name="base_documents", embedding_function=embed_fn_openai)

Chunking the document the NRG 304 document as test

In [143]:
document = joblib.load(os.path.join(
	knowledge_base_pickled_path, 
	"NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL FUNCTION.docling"
))

### Stage 2: RAG Modeling

Once the vector database is populated, the system will retrieve relevant information from both nursing handbooks and personal study notes. The retrieval will be augmented using an LLM (Large Language Model) for more accurate and contextually rich responses.

#### Steps:

1. **Retrieve Relevant Information**:
   - When a query is input by the user (e.g., "What are the symptoms of diabetes?"), the system will retrieve the most relevant content from the vector database.
   - ChromaDB will find the sections of the nursing handbooks or personal study notes that are most similar to the query.

2. **Augment Response Using Deepseek LLM**:
   - The retrieved sections will be passed through the **Deepseek LLM**, which will generate a response based on the content of the handbooks and notes.
   - The LLM will combine the relevant information and format it into a coherent, accurate response tailored to the question.

In [144]:
chunker = HybridChunker(max_tokens=100)
chunk_iter = chunker.chunk(dl_doc=document)

for i, chunk in enumerate(chunk_iter):
	print(f"=== {i} ===")
	print(chunk.meta.export_json_dict())
	enriched_text = chunker.serialize(chunk=chunk)
	print(enriched_text)
	break

=== 0 ===

{
    'schema_name': 'docling_core.transforms.chunker.DocMeta',
    'version': '1.0.0',
    'doc_items': [
        {
            'self_ref': '#/tables/0',
            'parent': {'$ref': '#/body'},
            'children': [],
            'content_layer': 'body',
            'label': 'table',
            'prov': [
                {
                    'page_no': 1,
                    'bbox': {
                        'l': 19.69459342956543,
                        't': 521.74951171875,
                        'r': 280.4553527832031,
                        'b': 293.35418701171875,
                        'coord_origin': 'BOTTOMLEFT'
                    },
                    'charspan': [0, 0]
                }
            ]
        }
    ],
    'headings': ['BRIEF REVIEW OF THE ANATOMY OF GIT'],
    'origin': {
        'mimetype': 'application/pdf',
        'binary_hash': 7347013283832062728,
        'filename': 'NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL FUNCTION.pdf'
    }
}

BRIEF REVIEW OF THE ANATOMY OF GIT
Oral Cavity and Pharynx  ●  Injury/ loss or decay of teeth  ●  Atrophy of taste buds  ●  ↓ Saliva production  ●  
Reduced ptyalin and amylase in saliva, Age-Related Changes in the Gastrointestinal System.Implications = Difficulty
chewing and  swallowing. ●  ↓ Motility and emptying  ●  Weakened gag reflex  ●  ↓ Resting pressure of lower 
esophageal  sphincter

In [145]:
class Metadata(BaseModel):
	user_id: str
	filename: str
	heading: str
	page_no: int

class Records(BaseModel):
	documents: list[str] = []
	metadatas: list[Metadata] = []
	ids: list[str] = []

In [146]:
chunker = HybridChunker(max_tokens=1000)
chunk_iter = chunker.chunk(dl_doc=document)
document_chunks = list(chunk_iter)

records = Records()

# Lock for thread-safe appends
lock = threading.Lock()

def extract_and_store(chunk: DocChunk):
	enriched_text = chunker.serialize(chunk=chunk)
	chunk_id = nanoid.generate()
	doc_chunk_metadata = chunk.meta.export_json_dict()
	metadata = Metadata(
		user_id="jiya",
		filename=doc_chunk_metadata['origin']['filename'],
		heading=doc_chunk_metadata['headings'][0],
		page_no=doc_chunk_metadata['doc_items'][0]['prov'][0]['page_no']
	)

	with lock:
		records.documents.append(enriched_text)
		records.metadatas.append(metadata.model_dump())
		records.ids.append(chunk_id)

with concurrent.futures.ThreadPoolExecutor() as executor:
	executor.map(extract_and_store, document_chunks)

In [161]:
await collection_base_documents.add(
	documents=records.documents,
	metadatas=records.metadatas,
	ids=records.ids
)

In [200]:
query_result = await collection_base_documents.query(
    query_texts=["what are the stool studies"],
    n_results=5,
    where={"user_id": "jiya"},
)

In [201]:
print(query_result)

{
    'ids': [
        [
            'QdcI5dpYEl4aEWz_YuDRa',
            '50BiMhJHXRbVtM1SDsTk7',
            'P-JSoDhr4oWryFymuaYJR',
            'lK6M3k1oAWdlE5RTolCHH',
            'Wx31s5t-XcVzqePwNLjvL'
        ]
    ],
    'distances': [
        [0.5813929777069989, 0.9317403510614783, 0.9606079983486142, 0.9681307638032123, 0.9800254028454096]
    ],
    'embeddings': None,
    'metadatas': [
        [
            {
                'filename': 'NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL 
FUNCTION.pdf',
                'heading': 'A. Stool Studies',
                'page_no': 2,
                'user_id': 'jiya'
            },
            {
                'filename': 'NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL 
FUNCTION.pdf',
                'heading': '5. Changes in Bowel Habits and Stool Characteristics',
                'page_no': 1,
                'user_id': 'jiya'
            },
            {
                'filename': 'NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL 
FUNCTION.pdf',
                'heading': 'H. Manometry and Electrophysiologic Studies',
                'page_no': 4,
                'user_id': 'jiya'
            },
            {
                'filename': 'NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL 
FUNCTION.pdf',
                'heading': '(3) Anoscopy, Proctoscopy, and Sigmoidoscopy',
                'page_no': 4,
                'user_id': 'jiya'
            },
            {
                'filename': 'NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL 
FUNCTION.pdf',
                'heading': '(4) Small Bowel Studies',
                'page_no': 4,
                'user_id': 'jiya'
            }
        ]
    ],
    'documents': [
        [
            'A. Stool Studies\n● Basic Stool Exam\n● Quantitative Study\n● Fecal Occult Blood Testing\n● Fecal 
Immunologic Testing (FIT)\n● Stool DNA Testing',
            '5. Changes in Bowel Habits and Stool Characteristics\na. Bulky  greasy,  foamy  stools  that  are  
foul  in  odor  and may or may not float.\nb. Light-gray or clay-colored stool, caused by a decrease or absence of 
conjugated bilirubin.\nc. Stool with mucus threads or pus that may be visible on gross inspection of the stool.\nd.
Small,  dry,  rock-hard  masses  occasionally  streaked with blood.\ne. Loose, watery stool that may or may not be 
streaked with blood.',
            'H. Manometry and Electrophysiologic Studies\n● Rectal  sensory  function  studies  are  used  to  
evaluate  rectal sensory function and neuropathy.\n● Esophageal manometry is used to detect motility disorders of 
the esophagus and the upper and lower esophageal sphincter.\n● Gastroduodenal, small intestine, and colonic 
manometry procedures are used to evaluate delayed gastric emptying and gastric  and  intestinal  motility  
disorders  such  as  irritable  bowel syndrome or atonic colon.\n● Electrogastrography,  an  electrophysiologic  
study,  also  may  be performed  to  assess  gastric  motility  disturbances  and  can  be useful in detecting 
motor or nerve dysfunction in the stomach.\n● Defecography  measures  anorectal  function  and  is  performed with 
very thick barium paste instilled into the rectum. Fluoroscopy is  used  to  assess  the function of the rectum and
anal sphincter while the patient attempts to expel the barrium.',
            '(3) Anoscopy, Proctoscopy, and Sigmoidoscopy\n● Endoscopic  examination  of the anus, rectum, and 
sigmoid and descending  colon  is  used  to  evaluate  chronic  diarrhea,  fecal incontinence, ischemic colitis, 
and lower GI hemorrhage and to observe for ulceration, fissures, abscesses, tumors, polyps, or other pathologic 
processes.',
            '(4) Small Bowel Studies\n● Capsule  endoscopy  allows  the  noninvasive  visualization  of  the mucosa
throughout  the  entire  small  intestine.

In [197]:
documents = query_result["documents"][0]
metadatas = query_result["metadatas"][0]

for doc, metadata in zip(documents, metadatas):
	print(doc)
	print(f"Source: {metadata['filename']}, Page: {metadata['page_no']}")

A. Stool Studies
● Basic Stool Exam
● Quantitative Study
● Fecal Occult Blood Testing
● Fecal Immunologic Testing (FIT)
● Stool DNA Testing

Source: NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL FUNCTION.pdf, Page: 2

5. Changes in Bowel Habits and Stool Characteristics
a. Bulky  greasy,  foamy  stools  that  are  foul  in  odor  and may or may not float.
b. Light-gray or clay-colored stool, caused by a decrease or absence of conjugated bilirubin.
c. Stool with mucus threads or pus that may be visible on gross inspection of the stool.
d. Small,  dry,  rock-hard  masses  occasionally  streaked with blood.
e. Loose, watery stool that may or may not be streaked with blood.

Source: NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL FUNCTION.pdf, Page: 1

H. Manometry and Electrophysiologic Studies
● Rectal  sensory  function  studies  are  used  to  evaluate  rectal sensory function and neuropathy.
● Esophageal manometry is used to detect motility disorders of the esophagus and the upper and lower esophageal 
sphincter.
● Gastroduodenal, small intestine, and colonic manometry procedures are used to evaluate delayed gastric emptying 
and gastric  and  intestinal  motility  disorders  such  as  irritable  bowel syndrome or atonic colon.
● Electrogastrography,  an  electrophysiologic  study,  also  may  be performed  to  assess  gastric  motility  
disturbances  and  can  be useful in detecting motor or nerve dysfunction in the stomach.
● Defecography  measures  anorectal  function  and  is  performed with very thick barium paste instilled into the 
rectum. Fluoroscopy is  used  to  assess  the function of the rectum and anal sphincter while the patient attempts 
to expel the barrium.

Source: NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL FUNCTION.pdf, Page: 4

(3) Anoscopy, Proctoscopy, and Sigmoidoscopy
● Endoscopic  examination  of the anus, rectum, and sigmoid and descending  colon  is  used  to  evaluate  chronic 
diarrhea,  fecal incontinence, ischemic colitis, and lower GI hemorrhage and to observe for ulceration, fissures, 
abscesses, tumors, polyps, or other pathologic processes.

Source: NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL FUNCTION.pdf, Page: 4

(4) Small Bowel Studies
● Capsule  endoscopy  allows  the  noninvasive  visualization  of  the mucosa  throughout  the  entire  small  
intestine.  It  is  particularly useful in the evaluation of obscure GI bleeding
● Double-balloon enteroscopy has made it possible to visualize the mucosa of the entire small bowel as well as 
carry out diagnostic and therapeutic interventions

Source: NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL FUNCTION.pdf, Page: 4

#### Response augmenting with deepseek r1

### Stage 2: Agentic RAG System

The **Agentic Layer** adds an intelligent routing mechanism to the RAG system. Instead of simply retrieving data from a static source, the system will decide where to get the most relevant information based on the context of the query and whether the query pertains to the nursing handbooks or her personal study notes.

#### Steps:

0. **Incorporate Personal Study Notes (Transes)**:
   - Personal study notes (transes) are documents or annotations written by the user, capturing key insights, case studies, or specific experiences.
   - These notes will be embedded in the same way as the nursing handbooks and stored in the same vector database.

1. **Agent Routes the Query**:
   - When a user (nursing student) asks a question, the **agent** will decide whether to retrieve data from the nursing handbooks or her personal study notes.
   - The agent will consider:
     - The **context** of the question (general knowledge vs. personalized experiences).
     - Whether her **personal study notes** are more relevant (e.g., if she’s already written notes on a particular case or condition).
     - If the query is general, the agent will prioritize retrieving information from the **nursing handbooks**.

2. **Retrieve from Both Sources**:
   - The system retrieves information from both the **nursing handbooks** and **personal study notes** based on the agent’s decision.
   - If the question pertains to something she has written down in her personal notes, the agent will prioritize retrieving those, combining them with general knowledge from the handbooks.

3. **Augment Response Using Deepseek LLM**:
   - The retrieved content from both sources is fed to the **Deepseek LLM**, which will merge the insights and generate a complete response.
   - The LLM will ensure the answer is not only accurate but also personalized based on the notes, making it more useful and contextually rich.
